In [1]:
# Cell 1
import sqlite3
import pandas as pd

def query_db(sql_query, db_path='employees.db'):
    """Execute SQL query and return results as DataFrame"""
    conn = sqlite3.connect(db_path)
    try:
        result = pd.read_sql_query(sql_query, conn)
        return result
    except Exception as e:
        print(f"Error: {e}")
        return None
    finally:
        conn.close()

# Cell 2 - Test it
query_db("SELECT COUNT(*) as total_employees FROM employees")

# Cell 3 - Interactive queries
query_db("""
SELECT d.department_name, COUNT(e.employee_id) as headcount
FROM departments d
LEFT JOIN employees e ON d.department_id = e.department_id
GROUP BY d.department_name
ORDER BY headcount DESC
""")

,department_name,headcount
0,Engineering,112
1,Sales,111
2,Marketing,111
3,Executive,90
4,Operations,76


In [2]:
query_db("""
SELECT reportsto, 
         COUNT(employee_id) as headcount, 
         avg(age) as avg_age
FROM employees
         where reportsto is not null
GROUP BY 1
ORDER BY 1
""")


,reportsto,headcount,avg_age
0,1,75,44.480000
1,2,6,48.666667
2,5,7,44.714286
3,6,6,42.500000
4,10,4,42.500000
...,...,...,...
136,450,2,28.500000
137,459,1,31.000000
138,463,1,56.000000
139,465,2,44.500000


In [5]:
query_db("""select * from employees limit 3""")

,employee_id,firstname,lastname,reportsto,position,age,department_id,salary,hire_date
0,1,Tony,Buchanan,1.0,Data Analyst,60,5,47966.56,2017-08-04
1,2,Jessica,Espinoza,1.0,VP Engineering,50,1,115811.38,2022-03-06
2,3,Walter,Goodwin,NaN,CEO,36,1,218526.25,2019-01-04


In [8]:
query_db("""
SELECT firstname,
         lastname,
         reportsto, 
         position, 
         age,
         case when reportsto is null then 'None' else 'CEO' end
FROM employees
         where reportsto in (Null, 1)
GROUP BY 1
ORDER BY 1
""")

,firstname,lastname,reportsto,position,age,case when reportsto is null then 'None' else 'CEO' end
0,Alexander,Huynh,1,Junior Engineer,64,CEO
1,Amanda,Black,1,VP Marketing,63,CEO
2,Amber,Thomas,1,VP Marketing,30,CEO
3,Andrew,Dawson,1,VP Engineering,50,CEO
4,Ann,Shields,1,VP Marketing,46,CEO
...,...,...,...,...,...,...
58,Tommy,Bishop,1,VP Engineering,33,CEO
59,Tony,Buchanan,1,Data Analyst,60,CEO
60,Vincent,Henson,1,VP Sales,54,CEO
61,Wendy,Martin,1,VP Engineering,37,CEO


* 🔢 ROW_NUMBER()
- Assigns unique sequential numbers to each row
- No ties - every row gets a different number
- Sequential - 1, 2, 3, 4, 5, 6...
* 🏆 RANK()
- Assigns ranks with gaps when there are ties
- Handles ties by giving them the same rank
- Skips numbers after ties - 1, 2, 2, 4, 5, 6...
* 🥇 DENSE_RANK()
- Assigns ranks without gaps when there are ties
- Handles ties by giving them the same rank
- No skipped numbers - 1, 2, 2, 3, 4, 5...

In [9]:
# ROW_NUMBER() - Assigns unique sequential numbers to rows
print("🔢 ROW_NUMBER() Example:")
print("Ranking employees by salary with unique row numbers")
query_db("""
SELECT 
    firstname || ' ' || lastname AS employee_name,
    position,
    salary,
    ROW_NUMBER() OVER (ORDER BY salary DESC) AS salary_row_number
FROM employees 
WHERE salary IS NOT NULL
ORDER BY salary DESC
LIMIT 15
""")

🔢 ROW_NUMBER() Example:
Ranking employees by salary with unique row numbers


,employee_name,position,salary,salary_row_number
0,Beth Frederick,CEO,310246.15,1
1,Theodore Rivera,CEO,293837.73,2
2,Karen Rodriguez,CEO,287017.04,3
3,Kyle Berry,CEO,277478.82,4
4,Larry Lawson,CEO,264551.70,5
5,Debbie Wallace,CEO,260379.91,6
6,Andrew Gray,CEO,254623.94,7
7,Stephanie Reid,CEO,252717.53,8
8,Robert Hartman,VP Marketing,250439.91,9
9,Amanda Skinner,CEO,250196.87,10


In [10]:
# RANK() - Assigns ranks with gaps for ties
print("🏆 RANK() Example:")
print("Ranking employees by salary (with gaps for ties)")
query_db("""
SELECT 
    firstname || ' ' || lastname AS employee_name,
    position,
    salary,
    RANK() OVER (ORDER BY salary DESC) AS salary_rank
FROM employees 
WHERE salary IS NOT NULL
ORDER BY salary DESC
LIMIT 15
""")

🏆 RANK() Example:
Ranking employees by salary (with gaps for ties)


,employee_name,position,salary,salary_rank
0,Beth Frederick,CEO,310246.15,1
1,Theodore Rivera,CEO,293837.73,2
2,Karen Rodriguez,CEO,287017.04,3
3,Kyle Berry,CEO,277478.82,4
4,Larry Lawson,CEO,264551.70,5
5,Debbie Wallace,CEO,260379.91,6
6,Andrew Gray,CEO,254623.94,7
7,Stephanie Reid,CEO,252717.53,8
8,Robert Hartman,VP Marketing,250439.91,9
9,Amanda Skinner,CEO,250196.87,10


In [11]:
# DENSE_RANK() - Assigns ranks without gaps for ties
print("🥇 DENSE_RANK() Example:")
print("Dense ranking by salary (no gaps for ties)")
query_db("""
SELECT 
    firstname || ' ' || lastname AS employee_name,
    position,
    salary,
    DENSE_RANK() OVER (ORDER BY salary DESC) AS dense_salary_rank
FROM employees 
WHERE salary IS NOT NULL
ORDER BY salary DESC
LIMIT 15
""")

🥇 DENSE_RANK() Example:
Dense ranking by salary (no gaps for ties)


,employee_name,position,salary,dense_salary_rank
0,Beth Frederick,CEO,310246.15,1
1,Theodore Rivera,CEO,293837.73,2
2,Karen Rodriguez,CEO,287017.04,3
3,Kyle Berry,CEO,277478.82,4
4,Larry Lawson,CEO,264551.70,5
5,Debbie Wallace,CEO,260379.91,6
6,Andrew Gray,CEO,254623.94,7
7,Stephanie Reid,CEO,252717.53,8
8,Robert Hartman,VP Marketing,250439.91,9
9,Amanda Skinner,CEO,250196.87,10


In [12]:
# PARTITION BY - Ranking within groups
print("🏢 PARTITION BY Example:")
print("Ranking employees by salary within each department")
query_db("""
SELECT 
    d.department_name,
    e.firstname || ' ' || e.lastname AS employee_name,
    e.position,
    e.salary,
    ROW_NUMBER() OVER (PARTITION BY d.department_name ORDER BY e.salary DESC) AS dept_rank
FROM employees e
JOIN departments d ON e.department_id = d.department_id
WHERE e.salary IS NOT NULL
ORDER BY d.department_name, dept_rank
""")

🏢 PARTITION BY Example:
Ranking employees by salary within each department


,department_name,employee_name,position,salary,dept_rank
0,Engineering,Bernard Floyd,Director Operations,181189.76,1
1,Engineering,Shannon Cortez,Director Engineering,180783.25,2
2,Engineering,Albert Jimenez,Director Engineering,160324.67,3
3,Engineering,Wendy Becker,Director Engineering,157437.24,4
4,Engineering,Xavier Garcia,Director Engineering,155987.94,5
...,...,...,...,...,...
495,Sales,Marc Hickman,Junior Sales Rep,40543.90,107
496,Sales,Victor Martinez,Junior Sales Rep,38877.73,108
497,Sales,Mikayla Carter,Sales Rep,38308.68,109
498,Sales,Maria Rodriguez,Junior Sales Rep,38264.95,110


In [13]:
# NTILE() - Divides rows into buckets/quartiles
print("📊 NTILE() Example:")
print("Dividing employees into salary quartiles (4 buckets)")
query_db("""
SELECT 
    firstname || ' ' || lastname AS employee_name,
    position,
    salary,
    NTILE(4) OVER (ORDER BY salary) AS salary_quartile,
    CASE 
        WHEN NTILE(4) OVER (ORDER BY salary) = 1 THEN 'Bottom 25%'
        WHEN NTILE(4) OVER (ORDER BY salary) = 2 THEN 'Lower Middle 25%'
        WHEN NTILE(4) OVER (ORDER BY salary) = 3 THEN 'Upper Middle 25%'
        WHEN NTILE(4) OVER (ORDER BY salary) = 4 THEN 'Top 25%'
    END AS quartile_description
FROM employees 
WHERE salary IS NOT NULL
ORDER BY salary DESC
LIMIT 20
""")

📊 NTILE() Example:
Dividing employees into salary quartiles (4 buckets)


,employee_name,position,salary,salary_quartile,quartile_description
0,Beth Frederick,CEO,310246.15,4,Top 25%
1,Theodore Rivera,CEO,293837.73,4,Top 25%
2,Karen Rodriguez,CEO,287017.04,4,Top 25%
3,Kyle Berry,CEO,277478.82,4,Top 25%
4,Larry Lawson,CEO,264551.70,4,Top 25%
5,Debbie Wallace,CEO,260379.91,4,Top 25%
6,Andrew Gray,CEO,254623.94,4,Top 25%
7,Stephanie Reid,CEO,252717.53,4,Top 25%
8,Robert Hartman,VP Marketing,250439.91,4,Top 25%
9,Amanda Skinner,CEO,250196.87,4,Top 25%


In [14]:
# LAG() - Access previous row values
print("⬅️ LAG() Example:")
print("Compare each employee's salary to the previous employee's salary")
query_db("""
SELECT 
    firstname || ' ' || lastname AS employee_name,
    position,
    salary,
    LAG(salary, 1) OVER (ORDER BY salary DESC) AS previous_salary,
    salary - LAG(salary, 1) OVER (ORDER BY salary DESC) AS salary_difference
FROM employees 
WHERE salary IS NOT NULL
ORDER BY salary DESC
LIMIT 15
""")

⬅️ LAG() Example:
Compare each employee's salary to the previous employee's salary


,employee_name,position,salary,previous_salary,salary_difference
0,Beth Frederick,CEO,310246.15,NaN,NaN
1,Theodore Rivera,CEO,293837.73,310246.15,-16408.42
2,Karen Rodriguez,CEO,287017.04,293837.73,-6820.69
3,Kyle Berry,CEO,277478.82,287017.04,-9538.22
4,Larry Lawson,CEO,264551.70,277478.82,-12927.12
5,Debbie Wallace,CEO,260379.91,264551.70,-4171.79
6,Andrew Gray,CEO,254623.94,260379.91,-5755.97
7,Stephanie Reid,CEO,252717.53,254623.94,-1906.41
8,Robert Hartman,VP Marketing,250439.91,252717.53,-2277.62
9,Amanda Skinner,CEO,250196.87,250439.91,-243.04


In [15]:
# LEAD() - Access next row values
print("➡️ LEAD() Example:")
print("Compare each employee's salary to the next employee's salary")
query_db("""
SELECT 
    firstname || ' ' || lastname AS employee_name,
    position,
    salary,
    LEAD(salary, 1) OVER (ORDER BY salary DESC) AS next_salary,
    salary - LEAD(salary, 1) OVER (ORDER BY salary DESC) AS salary_gap_to_next
FROM employees 
WHERE salary IS NOT NULL
ORDER BY salary DESC
LIMIT 15
""")

➡️ LEAD() Example:
Compare each employee's salary to the next employee's salary


,employee_name,position,salary,next_salary,salary_gap_to_next
0,Beth Frederick,CEO,310246.15,293837.73,16408.42
1,Theodore Rivera,CEO,293837.73,287017.04,6820.69
2,Karen Rodriguez,CEO,287017.04,277478.82,9538.22
3,Kyle Berry,CEO,277478.82,264551.70,12927.12
4,Larry Lawson,CEO,264551.70,260379.91,4171.79
5,Debbie Wallace,CEO,260379.91,254623.94,5755.97
6,Andrew Gray,CEO,254623.94,252717.53,1906.41
7,Stephanie Reid,CEO,252717.53,250439.91,2277.62
8,Robert Hartman,VP Marketing,250439.91,250196.87,243.04
9,Amanda Skinner,CEO,250196.87,249264.09,932.78


In [16]:
# Complex example combining multiple window functions
print("🎯 COMPLEX WINDOW FUNCTIONS Example:")
print("Comprehensive salary analysis using multiple window functions")
query_db("""
SELECT 
    d.department_name,
    e.firstname || ' ' || e.lastname AS employee_name,
    e.position,
    e.salary,
    -- Ranking functions
    ROW_NUMBER() OVER (ORDER BY e.salary DESC) AS overall_rank,
    RANK() OVER (PARTITION BY d.department_name ORDER BY e.salary DESC) AS dept_rank,
    DENSE_RANK() OVER (ORDER BY e.salary DESC) AS dense_rank,
    -- Quartiles
    NTILE(4) OVER (ORDER BY e.salary) AS salary_quartile,
    -- Comparison to adjacent salaries
    LAG(e.salary, 1) OVER (ORDER BY e.salary DESC) AS prev_salary,
    LEAD(e.salary, 1) OVER (ORDER BY e.salary DESC) AS next_salary,
    -- Department statistics
    AVG(e.salary) OVER (PARTITION BY d.department_name) AS dept_avg_salary,
    MAX(e.salary) OVER (PARTITION BY d.department_name) AS dept_max_salary,
    MIN(e.salary) OVER (PARTITION BY d.department_name) AS dept_min_salary
FROM employees e
JOIN departments d ON e.department_id = d.department_id
WHERE e.salary IS NOT NULL
ORDER BY e.salary DESC
LIMIT 20
""")

🎯 COMPLEX WINDOW FUNCTIONS Example:
Comprehensive salary analysis using multiple window functions


,department_name,employee_name,position,salary,overall_rank,dept_rank,dense_rank,salary_quartile,prev_salary,next_salary,dept_avg_salary,dept_max_salary,dept_min_salary
0,Executive,Beth Frederick,CEO,310246.15,1,1,1,4,NaN,293837.73,195061.103556,310246.15,115811.38
1,Executive,Theodore Rivera,CEO,293837.73,2,2,2,4,310246.15,287017.04,195061.103556,310246.15,115811.38
2,Executive,Karen Rodriguez,CEO,287017.04,3,3,3,4,293837.73,277478.82,195061.103556,310246.15,115811.38
3,Executive,Kyle Berry,CEO,277478.82,4,4,4,4,287017.04,264551.70,195061.103556,310246.15,115811.38
4,Executive,Larry Lawson,CEO,264551.70,5,5,5,4,277478.82,260379.91,195061.103556,310246.15,115811.38
5,Executive,Debbie Wallace,CEO,260379.91,6,6,6,4,264551.70,254623.94,195061.103556,310246.15,115811.38
6,Executive,Andrew Gray,CEO,254623.94,7,7,7,4,260379.91,252717.53,195061.103556,310246.15,115811.38
7,Executive,Stephanie Reid,CEO,252717.53,8,8,8,4,254623.94,250439.91,195061.103556,310246.15,115811.38
8,Executive,Robert Hartman,VP Marketing,250439.91,9,9,9,4,252717.53,250196.87,195061.103556,310246.15,115811.38
9,Executive,Amanda Skinner,CEO,250196.87,10,10,10,4,250439.91,249264.09,195061.103556,310246.15,115811.38


In [17]:

# OVER clause with different ordering and partitioning
print("🔄 OVER Clause Variations:")
print("Different ways to use the OVER clause")
query_db("""
SELECT 
    d.department_name,
    e.firstname || ' ' || e.lastname AS employee_name,
    e.age,
    e.salary,
    -- Different OVER clause examples
    COUNT(*) OVER () AS total_employees,
    COUNT(*) OVER (PARTITION BY d.department_name) AS dept_employees,
    AVG(e.salary) OVER () AS company_avg_salary,
    AVG(e.salary) OVER (PARTITION BY d.department_name) AS dept_avg_salary,
    SUM(e.salary) OVER (ORDER BY e.salary ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) AS running_salary_total,
    ROW_NUMBER() OVER (PARTITION BY d.department_name ORDER BY e.age) AS age_rank_in_dept
FROM employees e
JOIN departments d ON e.department_id = d.department_id
WHERE e.salary IS NOT NULL
ORDER BY d.department_name, e.salary DESC
LIMIT 25
""")

🔄 OVER Clause Variations:
Different ways to use the OVER clause


,department_name,employee_name,age,salary,total_employees,dept_employees,company_avg_salary,dept_avg_salary,running_salary_total,age_rank_in_dept
0,Engineering,Bernard Floyd,29,181189.76,500,112,102191.38026,89328.469554,38756834.49,20
1,Engineering,Shannon Cortez,53,180783.25,500,112,102191.38026,89328.469554,38575644.73,78
2,Engineering,Albert Jimenez,29,160324.67,500,112,102191.38026,89328.469554,34357142.17,22
3,Engineering,Wendy Becker,44,157437.24,500,112,102191.38026,89328.469554,34196817.50,50
4,Engineering,Xavier Garcia,58,155987.94,500,112,102191.38026,89328.469554,33726068.40,101
5,Engineering,William Phillips,36,154341.42,500,112,102191.38026,89328.469554,32949023.86,32
6,Engineering,Jonathon Adams,51,153814.55,500,112,102191.38026,89328.469554,32640696.82,76
7,Engineering,Lori Patrick,28,150960.39,500,112,102191.38026,89328.469554,31726161.38,18
8,Engineering,Jeffrey Jones,47,149670.09,500,112,102191.38026,89328.469554,31124310.21,63
9,Engineering,Heidi Pena,26,144947.36,500,112,102191.38026,89328.469554,30386546.87,12


In [ ]:
# Performance analysis using window functions
print("⭐ Performance Analysis with Window Functions:")
print("Analyzing performance ratings using window functions")
query_db("""
SELECT 
    e.firstname || ' ' || e.lastname AS employee_name,
    d.department_name,
    pr.rating,
    pr.review_date,
    -- Performance ranking
    RANK() OVER (ORDER BY pr.rating DESC) AS performance_rank,
    DENSE_RANK() OVER (PARTITION BY d.department_name ORDER BY pr.rating DESC) AS dept_performance_rank,
    -- Performance quartiles
    NTILE(4) OVER (ORDER BY pr.rating DESC) AS performance_quartile,
    -- Comparison to others
    AVG(pr.rating) OVER () AS company_avg_rating,
    AVG(pr.rating) OVER (PARTITION BY d.department_name) AS dept_avg_rating,
    pr.rating - AVG(pr.rating) OVER (PARTITION BY d.department_name) AS rating_vs_dept_avg,
    -- Previous and next ratings
    LAG(pr.rating, 1) OVER (ORDER BY pr.rating DESC) AS prev_rating,
    LEAD(pr.rating, 1) OVER (ORDER BY pr.rating DESC) AS next_rating
FROM employees e
JOIN departments d ON e.department_id = d.department_id
JOIN performance_reviews pr ON e.employee_id = pr.employee_id
ORDER BY pr.rating DESC
LIMIT 30
""")

In [ ]:
# Practice cell - Try your own window function queries
print("🧪 Practice Window Functions:")
print("Write your own window function queries here!")

my_query = """
-- Example: Find top 3 employees by salary in each department
SELECT 
    d.department_name,
    e.firstname || ' ' || e.lastname AS employee_name,
    e.salary,
    RANK() OVER (PARTITION BY d.department_name ORDER BY e.salary DESC) AS dept_salary_rank
FROM employees e
JOIN departments d ON e.department_id = d.department_id
WHERE e.salary IS NOT NULL
  AND RANK() OVER (PARTITION BY d.department_name ORDER BY e.salary DESC) <= 3
ORDER BY d.department_name, dept_salary_rank;
"""

In [ ]:
# Practice Cell - GROUP BY and HAVING
print("🧪 Practice GROUP BY and HAVING:")
print("Try writing your own GROUP BY and HAVING queries!")

# Example template:
my_query = """
-- Template: Find [groups] where [conditions]
SELECT 
    -- grouping columns
    -- aggregate functions: COUNT(), AVG(), SUM(), MIN(), MAX()
FROM employees e
LEFT JOIN departments d ON e.department_id = d.department_id
WHERE -- row-level filters (before grouping)
GROUP BY -- columns to group by
HAVING -- group-level filters (after grouping)
ORDER BY -- sorting
"""

print("💡 Common HAVING use cases:")
print("• COUNT(*) > 5 (groups with more than 5 items)")
print("• AVG(salary) > 80000 (groups with high average)")
print("• MAX(salary) - MIN(salary) > 50000 (high variance)")
print("• SUM(salary) > 500000 (high total)")